In [1]:
import numpy
import pandas
import math

In [2]:
bikelanes = pandas.read_csv('SDOT_Bikelanes.csv')
bikelanes.head()

,OBJECTID,STREET_NAME,LENGTH_MILES,DATE_COMPLETED,EXISTING_FACILITY_TYPE,PLANNED_FACILITY_TYPE,CATALYTIC_PROJECTS,STATUS,NETWORK,COMPKEY,PROJECT_NUMBER,DATE_PLANNED,DATE_EXPIRED,Shape,Shape_Length
0,1,RENTON AVE S,0.093986,12/31/2016 12:00:00 AM +0000,"In Street, Minor Separation","In Street, Minor Separation",0.0,NaN,LOCAL,12551.0,698.0,NaN,NaN,"(47.50067928200008, -122.24993698999998)",496.245732
1,2,21ST AVE W,0.189968,NaN,Sharrow,Neighborhood Greenway,0.0,NaN,CITYWIDE,3212.0,181.0,NaN,NaN,"(47.64201165700007, -122.38328161199996)",1003.033029
2,3,VALLEY ST,0.068389,NaN,"In Street, Minor Separation","In Street, Major Separation",0.0,UPGRADE,CITYWIDE,13267.0,384.0,NaN,NaN,"(47.62578474600008, -122.33581704399995)",361.091874
3,4,39TH AVE NE,0.119852,12/31/2012 12:00:00 AM +0000,Neighborhood Greenway,NaN,0.0,MEETS,CITYWIDE,5986.0,0.0,NaN,NaN,"(47.666840335000074, -122.28553618199999)",632.657676
4,5,2ND AV ET S,0.068258,12/31/2016 12:00:00 AM +0000,"In Street, Major Separation","In Street, Major Separation",0.0,UPGRADE,CITYWIDE,2926.0,215.0,NaN,NaN,"(47.60087849000007, -122.33121486499999)",360.403216


In [4]:
bikelanes = bikelanes.assign(BG_geoid = 'Nan')
bikelanes = bikelanes.assign(BG_label = 'Nan')

In [5]:
from shapely.geometry import shape, Point
import json
with open('BG.geojson') as f:
    js = json.load(f)

In [8]:
#for every bikelane data point (row in bikelanes), find which BG it is in (according to shape) 
#saved to file -- do not rerun ! 

count = 0
for ind, row in bikelanes.iterrows():
    if row.isnull()['Shape']:
        continue
    parse = row['Shape'].split(', ')
    lat = parse[0][1:]
    long = parse[1][:-1]
    #print(lat,long)
    point = Point(float(long), float(lat))
    for feature in js['features']:
        #print(feature)
        polygon = shape(feature['geometry'])
        #print(polygon.contains(point))
        if polygon.contains(point):
            count = count + 1
            found_polygon = feature['properties']
            #print(polygon['geoid'], polygon['name'])
            bikelanes.loc[ind, 'BG_geoid'] = found_polygon['GEO_ID_GRP']
            bikelanes.loc[ind, 'BG_label'] = found_polygon['BLKGRP_LBL']
            #print('Found containing polygon:', feature['properties'])
            break

In [15]:
bikelanes.head(10)

,OBJECTID,STREET_NAME,LENGTH_MILES,DATE_COMPLETED,EXISTING_FACILITY_TYPE,PLANNED_FACILITY_TYPE,CATALYTIC_PROJECTS,STATUS,NETWORK,COMPKEY,PROJECT_NUMBER,DATE_PLANNED,DATE_EXPIRED,Shape,Shape_Length,BG_geoid,BG_label
0,1,RENTON AVE S,0.093986,12/31/2016 12:00:00 AM +0000,"In Street, Minor Separation","In Street, Minor Separation",0.0,NaN,LOCAL,12551.0,698.0,NaN,NaN,"(47.50067928200008, -122.24993698999998)",496.245732,530330119002,Blkgrp 0119002
1,2,21ST AVE W,0.189968,NaN,Sharrow,Neighborhood Greenway,0.0,NaN,CITYWIDE,3212.0,181.0,NaN,NaN,"(47.64201165700007, -122.38328161199996)",1003.033029,530330058023,Blkgrp 0058023
2,3,VALLEY ST,0.068389,NaN,"In Street, Minor Separation","In Street, Major Separation",0.0,UPGRADE,CITYWIDE,13267.0,384.0,NaN,NaN,"(47.62578474600008, -122.33581704399995)",361.091874,530330066001,Blkgrp 0066001
3,4,39TH AVE NE,0.119852,12/31/2012 12:00:00 AM +0000,Neighborhood Greenway,NaN,0.0,MEETS,CITYWIDE,5986.0,0.0,NaN,NaN,"(47.666840335000074, -122.28553618199999)",632.657676,530330042004,Blkgrp 0042004
4,5,2ND AV ET S,0.068258,12/31/2016 12:00:00 AM +0000,"In Street, Major Separation","In Street, Major Separation",0.0,UPGRADE,CITYWIDE,2926.0,215.0,NaN,NaN,"(47.60087849000007, -122.33121486499999)",360.403216,530330092002,Blkgrp 0092002
5,6,5TH AVE,0.043032,12/31/2016 12:00:00 AM +0000,Sharrow,"In Street, Major Separation",0.0,NaN,CITYWIDE,7223.0,316.0,NaN,NaN,"(47.60168377100007, -122.32765594099999)",227.208470,530330092001,Blkgrp 0092001
6,7,5TH AVE,0.057913,12/31/2016 12:00:00 AM +0000,Sharrow,"In Street, Major Separation",0.0,NaN,CITYWIDE,7225.0,316.0,NaN,NaN,"(47.60298463200007, -122.32853488999996)",305.781989,530330081002,Blkgrp 0081002
7,8,20TH AVE S,0.057875,12/31/2009 12:00:00 AM +0000,Sharrow,Neighborhood Greenway,0.0,UPGRADE,CITYWIDE,3038.0,153.0,NaN,NaN,"(47.60083481500004, -122.30635358499995)",305.579532,530330090002,Blkgrp 0090002
8,9,RENTON AVE S,0.046922,12/31/2012 12:00:00 AM +0000,"In Street, Minor Separation","In Street, Minor Separation",0.0,NaN,LOCAL,12520.0,699.0,NaN,NaN,"(47.52855283700006, -122.27794189999997)",247.746301,530330117001,Blkgrp 0117001
9,10,LATONA AVE NE,0.051113,12/31/2012 12:00:00 AM +0000,"In Street, Minor Separation","In Street, Minor Separation",0.0,NaN,LOCAL,11304.0,535.0,NaN,NaN,"(47.67009744100005, -122.32554278799995)",269.876785,530330045002,Blkgrp 0045002


In [10]:
bikelanes.to_csv('bikelanes_withBG.csv')

In [11]:
total_len = bikelanes.groupby('BG_geoid').sum()['Shape_Length']
total_len.head()

BG_geoid
530330001001    4536.719056
530330001002    1412.992527
530330001003    4248.356051
530330001004    1687.041064
530330001005     918.755231
Name: Shape_Length, dtype: float64

In [12]:
total_len.to_csv('BG_totalBL.csv')

In [22]:
#BG bikelane length where type/rank = something
result1 = bikelanes.loc[bikelanes['EXISTING_FACILITY_TYPE'] == 'Multi-use Trail']
result11 = result1.groupby(['BG_geoid']).sum()['Shape_Length']
result11.head()

BG_geoid
530330001001     1873.740358
530330004022     5044.330565
530330006005      236.028363
530330009001     1213.035312
530330009002    10323.361805
Name: Shape_Length, dtype: float64

In [23]:
result11.to_csv('BG_BL_MU_trail.csv')